### RL tutorial. Part 2.

In this tutorial we will learn how implement different agents that keep track of the value of their actions and the state they are in. We will use 4 different agents:
- Random agent.
- SARSA agent.
- Q-learning agent.
- Q-learning agent with eligibility traces.

### GridWorld

We will train our agents to navigate a gridworld. The gridworld is a 2D grid of size 8x8. The agent can move in 4 directions: up, down, left and right, or stay. The agent receives a reward of -1 for each step it takes, a reward of 50 if it reaches the goal state and a reward of -2 if it hits an obstacle. 

The gridworld will be implemented with a class with the following methods:
- `__init__`: initializes the gridworld with a given size, a list of obstacles and a goal state (check the other input parameters).
- `new_trial`: resets the state of the agent to the start position and the count of steps within the trial.
- `step`: takes an action as input and returns the next state, the reward and a dictionary with a boolean indicating if the trial is finished.
- `render`: prints the gridworld with the current position of the agent.


In [ ]:
# import the necessary packages
import numpy as np
from gym import spaces
import matplotlib.pyplot as plt
# from copy import deepcopy


# We now create a class that will contain the task. 
# This class will be called to generate the trials that we will use to train the agent.
class GridWorld():
    """
     GridWorld task.

    Args:
        size: int, size of the grid
        start: tuple, starting position of the agent
        goal: tuple, goal position of the agent
        obstacles: list of tuples, positions of the obstacles
    """
    def __init__(self, size=5, start=(0, 0), goal=(4, 4), obstacles=None,
                 max_steps_per_trial=50):
        # define the task parameters
        self.size = size
        self.start = start
        self.goal = goal
        self.obstacles = obstacles or []
        self.max_steps_per_trial = max_steps_per_trial
        # observation space
        self.observation_space = spaces.Box(-np.inf, np.inf,
                                            shape=(9,), dtype=np.float32)
        # action space
        self.action_space = spaces.Discrete(5)
        # rewards for reaching the goal (correct), not reaching the goal (fail) and hitting an obstacle
        self.rewards = {'correct': 50, 'obstacle': -2, 'fail': -1}
        self.position = self.start
        # step within the trial
        self.t = 0
        self.trial = 0

    def new_trial(self, **kwargs):
        """
        new_trial() is called when a trial ends to generate the next trial.
        kwargs could be used to modify the goal or start positions.    
        """
        self.t = 0
        self.trial += 1
        self.position = self.start

    def step(self, action):
        """
        step receives an action and returns:
            a new observation, the position of the agent
            reward associated with the action, reward
            a dictionary with extra information:
                boolean indicating the end of the trial, info['new_trial']
        """
        self.t += 1
        new_trial = False
        candidate = self.position
        # process action
        if action == 1:  # Move up
            candidate = (max(self.position[0] - 1, 0), self.position[1])
        elif action == 2:  # Move right
            candidate = (self.position[0], min(self.position[1] + 1, self.size - 1))
        elif action == 3:  # Move down
            candidate = (min(self.position[0] + 1, self.size - 1), self.position[1])
        elif action == 4:  # Move left
            candidate = (self.position[0], max(self.position[1] - 1, 0))
        
        # Updtate reward and position
        if candidate == self.goal:
            reward = self.rewards['correct']
            self.position = self.start
            new_trial = True
            self.new_trial()
        elif candidate in self.obstacles:
            reward = self.rewards['obstacle']
        else:
            self.position = candidate
            reward = self.rewards['fail']

        # check time limit
        if self.t > self.max_steps_per_trial:
            new_trial = True
            self.new_trial()

        return self.position, reward, {'new_trial': new_trial}

    def render(self, ax=None):
        """
        Display position of the agent, obstacles and goal in a grid.
        """
        grid = np.zeros((self.size, self.size))
        grid[self.position] = 1  # agent's position marked with 1
        grid[self.goal] = 2  # goal position marked with 0.5
        for obstacle in self.obstacles:
            grid[obstacle] = -1
        # pad grid with -1
        grid = np.pad(grid, pad_width=1, mode='constant', constant_values=-1)
        if ax is None:
            _, ax = plt.subplots()
        ax.imshow(grid, vmin=-1, vmax=2, cmap='hot')

##### Define gridworld parameters

In [ ]:
grid_size = 5
start = (0, 0)
goal = (grid_size - 1, grid_size - 1)
obstacles = [(x, grid_size//2) for x in range(grid_size-2)]
max_steps_per_trial = 20

##### Plot an example environment

In [ ]:

env = GridWorld(size=grid_size, start=start, goal=goal, obstacles=obstacles, max_steps_per_trial=max_steps_per_trial)
actions = ['stay', 'up', 'right', 'down', 'left']
good_path = [3, 2, 3, 2, 3, 2, 3, 3, 3, 2, 2, 2, 3, 2]
f, ax = plt.subplots(figsize=(10, 10), nrows=3, ncols=5)
ax = ax.flatten()
for i_a, action in enumerate(good_path):
    env.step(action)
    env.render(ax=ax[i_a])
    # remove xtiks and yticks
    ax[i_a].set_xticks([])
    ax[i_a].set_yticks([])
    # add title
    ax[i_a].set_title(actions[action])
# remove last axis
ax[-1].axis('off')

### Plotting functions

In [ ]:
def plot_reward_per_episode(reward_per_episode, ax=None):   
    # Plot rewards
    if ax is None:
        _, ax = plt.subplots(figsize=(6, 6))
        ax.set_title("Reward per episode")
        ax.set_xlabel("Episode")
        ax.set_ylabel("Total Reward")
    ax.plot(reward_per_episode)
    # plt.show()

def plot_q_table(agent, action_space):
    # plot q_table
    f, ax = plt.subplots(ncols=2, nrows=3, figsize=(10, 10))
    ax = ax.flatten()
    # actions
    titles = ['Stay', 'Up', 'Right', 'Down', 'Left']
    for i_a, a in enumerate(ax):
        if i_a < len(action_space):
            im = a.imshow(agent.get_q_table()[:, :, i_a], cmap='hot')
            plt.colorbar(im, ax=a)
            a.set_title(titles[i_a])
    plt.show()

### Training function

This is the function we will use to train all our agents. Think of what you need from the agent to update the gridworld and what the agent needs from the gridworld to act and learn to make better decisions.

In [ ]:
def train_agent(agent, grid_size=5, start=(0, 0), goal=(4, 4), obstacles=[(x, 3) for x in range(3)],
                max_num_tr=100, verbose=False, max_steps_per_trial=50):
    """
    Create gridWorld and agent, and train agent for a number of trials.

    Args:
        agent: agent to train
        grid_size: int, size of the grid
        start: tuple, starting position of the agent
        goal: tuple, goal position for the agent
        obstacles: list of tuples, positions of the obstacles
        max_num_tr: int, maximum number of trials
        verbose: bool, whether to display the environment
        max_steps_per_trial: int, maximum number of steps per trial
    """
    # Create grid world
    env = GridWorld(size=grid_size, start=start, goal=goal, obstacles=obstacles, max_steps_per_trial=max_steps_per_trial)

    # INSTRUCTION 1: first action of the agent. Which parameters do you need to pass to the agent?
    action = agent.get_action(state=start)
    state, _, _ = env.step(action)
    reward_per_episode = []
    total_reward = 0
    while env.trial < max_num_tr:
        # INSTRUCTION 2: get the next action from the agent.
        action = agent.get_action(state=state)
        next_state, reward, info = env.step(action)
        # INSTRUCTION 3: learn from the experience. Which parameters do you need to pass to the agent?
        agent.learn(state=state, action=action, reward=reward, next_state=next_state)
        state = next_state
        if verbose:
            print(f"Trial: {env.trial + 1}, Step: {env.t + 1}, State: {state}, Action: {action}, Next State: {next_state},\
                    Reward: {reward}, Total Reward: {total_reward}, New trial: {info['new_trial']}")
        total_reward += reward
        if info['new_trial']: # check if the trial has ended
            reward_per_episode.append(total_reward)
            total_reward = 0
    return reward_per_episode

### Random Agent

The random agent does not care about the environment and choses an action from the available ones.

In [ ]:
class Random: 
    def __init__(self, action_space):
        self.action_space = action_space
    # INSTRUCTION 4: return the agent's action.
    # (hint: you might need to add some dummy input parameters (i.e. not used) so the function works well with the train_agent function)
    def get_action(self, **kwargs):  # TODO: explain kwargs
        return np.random.choice(self.action_space)
    # INSTRUCTION 5: learn from experience (if needed)
    # (hint: you might need to add some dummy input parameters (i.e. not used) so the function works well with the train_agent function)
    def learn(self, **kwargs): 
        pass

In [ ]:
action_space = [0, 1, 2, 3, 4]  # up, right, down, left
max_num_tr = 1000
verbose = True
agent = Random(action_space=action_space)
reward_per_episode_random =\
      train_agent(agent=agent, grid_size=grid_size, start=start, goal=goal,
                  obstacles=obstacles, max_num_tr=max_num_tr,
                  verbose=verbose, max_steps_per_trial=max_steps_per_trial)
plot_reward_per_episode(reward_per_episode_random)

You see there random agent does not do great, but its performance will be a good baseline for us to evaluate the learning of the rest of the agents.

### SARSA Agent

The state-action-reward-state-action (SARSA) agent uses an on-line policy to learn the best policy. The SARSA algorithm works by maintaining a table of action-value estimates Q(s, a) (the q_table), where s is the state and a is the action taken by the agent in that state. 

In [ ]:
class SARSAAgent:
    def __init__(self, action_space, state_space, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.action_space = action_space
        self.state_space = state_space
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = np.random.rand(self.state_space[0], self.state_space[1], len(self.action_space))
    # INSTRUCTION 6: return the agent's action.
    def get_action(self, state):
        # INSTRUCTION 6.1: implement epsilon-greedy policy
        if state is None or np.random.rand() < self.epsilon:  # Explore: select a random action
            return np.random.choice(self.action_space)
        else:                                # Exploit: select the action with max value (greedy policy)
            return np.argmax(self.q_table[state[0], state[1]])
    # INSTRUCTION 7: learn from experience
    def learn(self, state, action, reward, next_state):
        # INSTRUCTION 7.1: get old value from the q-table
        old_value = self.q_table[state[0], state[1], action]
        # INSTRUCTION 7.2: get action from the q-table
        next_action = self.get_action(next_state)
        # INSTRUCTION 7.3: get the maximum value from the q-table for the next state and action
        next_max = self.q_table[next_state[0], next_state[1], next_action]
        # INSTRUCTION 7.4: update the q-table
        new_value = (1 - self.alpha) * old_value + self.alpha * (reward + self.gamma * next_max)
        self.q_table[state[0], state[1], action] = new_value
    def get_q_table(self):
        return self.q_table

In [ ]:
state_space = (grid_size, grid_size)
epsilon = 0.1
agent = SARSAAgent(action_space=action_space, state_space=state_space, epsilon=epsilon)
reward_per_episode_sarsa =\
      train_agent(agent=agent, grid_size=grid_size, start=start, goal=goal,
                  obstacles=obstacles, max_num_tr=max_num_tr,
                  verbose=verbose, max_steps_per_trial=max_steps_per_trial)
plot_reward_per_episode(reward_per_episode_sarsa)
plot_q_table(agent, action_space)

### Q-Learning Agent

The Q-learning agent uses an off-line policy to learn about the environment. As the SARSA algorithm, it works by maintaining a table of action-value estimates Q(s, a). The difference between the two is in the way they update the q-table. 

In [ ]:
class QLearningAgent:
    def __init__(self, action_space, state_space, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.action_space = action_space
        self.state_space = state_space
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = np.random.rand(self.state_space[0], self.state_space[1], len(self.action_space))
    # INSTRUCTION 8: return the agent's action.
    def get_action(self, state):
        # INSTRUCTION 8.1: implement epsilon-greedy policy
        if state is None or np.random.rand() < self.epsilon:  # Explore: select a random action
            return np.random.choice(self.action_space)
        else:                                # Exploit: select the action with max value (greedy policy)
            return np.argmax(self.q_table[state[0], state[1]])
    # INSTRUCTION 9: learn from experience
    def learn(self, state, action, reward, next_state):
        # INSTRUCTION 9.1: get old value from the q-table
        old_value = self.q_table[state[0], state[1], action]
        # INSTRUCTION 9.2: get the maximum value from the q-table for the next state and action. This is different from SARSA!
        next_max = np.max(self.q_table[next_state[0], next_state[1]])
        # INSTRUCTION 9.3: update the q-table
        new_value = (1 - self.alpha) * old_value + self.alpha * (reward + self.gamma * next_max)
        self.q_table[state[0], state[1], action] = new_value
    def get_q_table(self):
        return self.q_table

In [ ]:
state_space = (grid_size, grid_size)
epsilon = 0.1
agent = QLearningAgent(action_space=action_space, state_space=state_space, epsilon=epsilon)

reward_per_episode_ql =\
      train_agent(agent=agent, grid_size=grid_size, start=start, goal=goal,
                  obstacles=obstacles, max_num_tr=max_num_tr,
                  verbose=verbose, max_steps_per_trial=max_steps_per_trial)
plot_reward_per_episode(reward_per_episode_ql)
plot_q_table(agent, action_space)

### Q-Learning Agent with elegibility trace

In [ ]:
class QLearningAgent_with_eligibility:
    def __init__(self, action_space, state_space, alpha=0.1, gamma=0.9, epsilon=0.1, lambda_=0.9):
        self.action_space = action_space
        self.state_space = state_space
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.lambda_ = lambda_
        self.q_table = np.random.rand(self.state_space[0], self.state_space[1], len(self.action_space))
        self.eligibility_trace = np.zeros((self.state_space[0], self.state_space[1]))
    # INSTRUCTION 10: return the agent's action.
    def get_action(self, state):
        if state is None or np.random.rand() < self.epsilon:  # Explore: select a random action
            return np.random.choice(self.action_space)
        else:  # Exploit: select the action with max value (greedy policy)
            return np.argmax(self.q_table[state[0], state[1]])
    # INSTRUCTION 11: learn from experience
    def learn(self, state, action, reward, next_state):
        # INSTRUCTION 11.1: get old value from the q-table
        old_value = self.q_table[state[0], state[1], action]
        # INSTRUCTION 11.2: update eligibility trace
        self.eligibility_trace *= self.gamma * self.lambda_
        # INSTRUCTION 11.3: update eligibility trace for the current state and action
        self.eligibility_trace[state[0], state[1]] += 1
        # INSTRUCTION 11.4: get the maximum value from the q-table for the next state and action        
        next_max = np.max(self.q_table[next_state[0], next_state[1]])

        # INSTRUCTION 11.5: calculate the temporal difference delta
        delta = reward + self.gamma * next_max - old_value

        # INSRTUCTION 11.6: Update Q-value using eligibility trace
        self.q_table[state[0], state[1], action] += self.alpha * delta * self.eligibility_trace[state[0], state[1]]
    def get_q_table(self):
        return self.q_table


In [ ]:
state_space = (grid_size, grid_size)
epsilon = 0.1
alpha = 0.1
gamma = 0.9 
lambda_= 0.9
agent = QLearningAgent_with_eligibility(action_space=action_space, state_space=state_space, epsilon=epsilon,
                                        alpha=alpha, gamma=gamma, lambda_=lambda_)
reward_per_episode_et =\
      train_agent(agent=agent, grid_size=grid_size, start=start, goal=goal,
                  obstacles=obstacles, max_num_tr=max_num_tr,
                  verbose=verbose, max_steps_per_trial=max_steps_per_trial)
plot_reward_per_episode(reward_per_episode_et)
plot_q_table(agent, action_space)

### Compare learning

In [ ]:
# create figure
f, ax = plt.subplots(figsize=(6, 6))
plot_reward_per_episode(reward_per_episode_random, ax=ax)
plot_reward_per_episode(reward_per_episode_sarsa, ax=ax)
plot_reward_per_episode(reward_per_episode_ql, ax=ax)
plot_reward_per_episode(reward_per_episode_et, ax=ax)
# add legend to ax
ax.legend(['Random', 'SARSA', 'Q-Learning', 'Q-Learning with Eligibility Trace'])
ax.set_title("Reward per episode")
ax.set_xlabel("Episode")
ax.set_ylabel("Total Reward")

